Moodle XML Question Generator, (C) 2019-2020 Kosaku Nagasaka (Kobe University)

# moodle_xqgでのグラフ問題の作り方

Moodle XML Question Generator の Python 版である moodle_xqg で，グラフ問題を作る方法を説明します。

## パッケージの読み込み

**moodle_xqg.core**は必ず必要です。**moodle_xqgb.qbank.common**は数式のLaTeX化など，問題作成に必要なものが含まれているので，読み込んだほうがよいです。

In [1]:
import moodle_xqg.core as mxqg
import moodle_xqg.qbank.common as mxqg_common

次のパッケージは作られる問題に依存しますが，多くの場合，乱数生成と数式処理のために，必要となると思います。

In [2]:
import random
import sympy

## グラフの問題のサンプル

問題生成器は，**Question**の小クラスとして定義する必要があります。必要なメソッドは，**question_generate**と**correct_answers_generate**と**incorrect_answers_generate**と**question_text**と**answer_text**の5つです。

問題生成器は，**Quiz.quiz_identifier**に同じ問題であるかを判定可能なデータを必ず格納してください。この値でもって問題の区別をするので，格納していない場合，最初の1問しか自動生成されなくなってしまいます。 

グラフをSVGとして生成している部分は **_make_svg_str** になります。

In [3]:
class linear_polynomial_graph(mxqg.Question):
    def __init__(self, lower=-10, upper=10):
        # 生成する整数の範囲を指定できるように修正
        self.lower = lower
        self.upper = upper
        self.x = sympy.symbols('x')
        self.correct_f = 0
    def question_generate(self, _quiz_number=0):
        quiz = mxqg.Quiz(name='1次関数のグラフ', quiz_number=_quiz_number, lang='ja')
        # 問題データの生成（SymPyは，0を計算せずにLaTeXへは変換できないので，0を除外してる）
        quiz.data = [random.choice(list(set(range(self.lower, self.upper + 1))-{0})) for i in range(2)]
        quiz.quiz_identifier = hash('{}*self.x+{}'.format(quiz.data[0], quiz.data[1]))
        # 正答の選択肢の生成
        self.correct_f = quiz.data[0]*self.x+quiz.data[1]
        ans = { 'fraction': 100, 'data': mxqg_common.sympy_expr_to_svg(self.correct_f) }
        quiz.answers.append(ans)
        return quiz        
    def correct_answers_generate(self, quiz, size=1):
        # 正答を個別には作らないので何もしない
        pass
    def incorrect_answers_generate(self, quiz, size=4):
        answers = []
        ans = { 'fraction': 0 }
        # 傾き符号違い
        f = -quiz.data[0]*self.x+quiz.data[1]
        if f != self.correct_f:
            ans['data'] = mxqg_common.sympy_expr_to_svg(f)
            ans['feedback'] = '直線の傾きの正負が異なります。'
            answers.append(dict(ans))
       # 切片符号違い
        f = quiz.data[0]*self.x-quiz.data[1]
        if f != self.correct_f:
            ans['data'] = mxqg_common.sympy_expr_to_svg(f)
            ans['feedback'] = '切片は，直線とy軸との交点になります。'
            answers.append(dict(ans))
        # 傾きと切片符号違い
        f = -quiz.data[0]*self.x-quiz.data[1]
        if f != self.correct_f:
            ans['data'] = mxqg_common.sympy_expr_to_svg(f)
            ans['feedback'] = '傾きも切片もどちらも復習してください。'
            answers.append(dict(ans))
        # 切片違い
        f = quiz.data[0]*(self.x + quiz.data[1] - 
                          sympy.Rational(quiz.data[1],quiz.data[0]))+quiz.data[1]
        if f != self.correct_f:
            ans['data'] = mxqg_common.sympy_expr_to_svg(f)
            ans['feedback'] = '切片は，直線とy軸との交点になります。'
            answers.append(dict(ans))
        if len(answers) >= size:
            return random.sample(answers,k=size)
        return answers    
    def question_text(self, quiz):
        return '次の関数 \( f(x) \) に関して， \( y=f(x) \) のグラフを選択せよ。<br />' + \
                mxqg_common.sympy_expr_to_text(quiz.data[0]*self.x+quiz.data[1], prefix='f(x)=')
    def answer_text(self, ans):
        return ans['data']

問題の自動生成は，**generate**を使いますが，それに先立ち，問題生成器のインスタンスを生成しておく必要があります。作成する問題数は，*generate*に**size=100**のような形で指定します（無指定時は，10個生成されます）。

In [4]:
lpg = linear_polynomial_graph()
lpg_quizzes = mxqg.generate(lpg, category='グラフの問題のサンプル')

生成結果は，**Quizzes**のインスタンスとなっており，次のようにプレビューができます。

In [5]:
lpg_quizzes.preview(size=2)

このようにして生成した結果は，**save**を使ってMoodleにインポート可能なファイルに保存することができます。

In [6]:
lpg_quizzes.save('linear_polynomial_graph_in_japanese.xml')

実際にインポートして確認してみてください。なお，問題整理番号の表示が不要な場合は，**show_quiz_number=False**をオプションとして*save*に付けてください。